In [1]:
import urllib.request
import zipfile

In [5]:
url="https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"

In [6]:
file_name="horse-or-human.zip"
training_dir='horse-or-human/training/'
urllib.request.urlretrieve(url=url,filename=file_name)

('horse-or-human.zip', <http.client.HTTPMessage at 0x7f6ad20b57b8>)

In [7]:
zip_ref=zipfile.ZipFile(file_name,'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator=train_datagen.flow_from_directory(training_dir,target_size=(300,300),class_mode='binary')

Found 1027 images belonging to 2 classes.


In [10]:
import tensorflow as tf


In [16]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [18]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 33, 33, 64)       

In [23]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])


In [24]:
validation_url="https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name="validation-horse-or-human.zip"
validation_dir='horse-or-human/validation/'
urllib.request.urlretrieve(validation_url,validation_file_name)

('validation-horse-or-human.zip', <http.client.HTTPMessage at 0x7f6a8c3bcf98>)

In [25]:
zip_ref=zipfile.ZipFile(validation_file_name,'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [26]:
validation_datagen=ImageDataGenerator(rescale=1/255)
validation_generator=train_datagen.flow_from_directory(validation_dir,target_size=(300,300),class_mode='binary')

Found 256 images belonging to 2 classes.


In [27]:
history=model.fit_generator(train_generator,epochs=15,validation_data=validation_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
33/33 [==============================] - 92s 3s/step - loss: 0.7286 - accuracy: 0.5839 - val_loss: 0.5714 - val_accuracy: 0.8438
Epoch 2/15
33/33 [==============================] - 90s 3s/step - loss: 0.4116 - accuracy: 0.8418 - val_loss: 0.8236 - val_accuracy: 0.8594
Epoch 3/15
33/33 [==============================] - 90s 3s/step - loss: 0.2347 - accuracy: 0.9155 - val_loss: 0.9467 - val_accuracy: 0.8320
Epoch 4/15
33/33 [==============================] - 90s 3s/step - loss: 0.0865 - accuracy: 0.9701 - val_loss: 0.8725 - val_accuracy: 0.7773
Epoch 5/15
33/33 [==============================] - 90s 3s/step - loss: 0.0686 - accuracy: 0.9820 - val_loss: 2.0325 - val_accuracy: 0.8320
Epoch 6/15
33/33 [==============================] - 91s 3s/step - loss: 0.1139 - accuracy: 0.9682 - val_loss: 1.7917 - val_accuracy: 0.8203
Epoch 7/15
33/33 [==============================] - 90s 3s/step - loss: 0.2384 - accuracy: 0.9429 - val_loss: 1.6251 - val_accuracy: 0.8281
Epoch 8/15
33/33 [==

In [32]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

upload=files.upload()
for fn in upload.keys():
  path='/content/'+fn
  img=image.load_img(path,target_size=(300,300))
  X=image.img_to_array(img)
  X=np.expand_dims(X,axis=0)

  image_tensor=np.vstack([X])
  classes=model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn+"is a human")
  else:
    print(fn+"is a horse")


Saving human01-00.png to human01-00.png
[[1.]]
[1.]
human01-00.pngis a human
